# Environment finder

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, widgets
import ase
from ase.io import read, write
from ase.neighborlist import NeighborList
from ase.visualize.plot import plot_atoms
from itertools import permutations
from tqdm.notebook import tqdm, trange
import nglview as nv
import warnings
import time
from os import listdir

warnings.simplefilter('ignore')

_ColormakerRegistry()

In [2]:
box_layout = widgets.Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='90%')

In [3]:
def plotConf():
    global conf
    #plt.rcParams['figure.figsize'] = [15, 7]
    #fig, ax = plt.subplots()
    #anglestr = str(anglex) + "x," + str(angley) + "y," + str(anglez) + "z"
    #plot_atoms(conf, ax, radii=0.3, rotation=(anglestr))
    #plt.show()
    v = nv.show_ase(conf)
    v.add_representation("unitcell")
    boxy=widgets.Box([v],layout=box_layout)
    display(boxy)

def chooseConfiguration(Configuration):
    global conf, atom_types
    filename=Configuration
    conf = read(filename)
    atom_types = np.unique(conf.get_chemical_symbols())

def chooseAndPlotConfiguration(Configuration):
    global conf, atom_types
    chooseConfiguration(Configuration)
    plotConf()
    #w = interactive(plotConf, anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
    #display(w)
    
Widget1UploadConfiguration = widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    wait=True
)

def toggleChooseAndUpload(value):
    if (value=='Choose'):
        # Check of file has been uploaded
        if (Widget1UploadConfiguration.value!={}):
            uploaded_filename = next(iter(Widget1UploadConfiguration.value))
            content = Widget1UploadConfiguration.value[uploaded_filename]['content']
            with open("Uploaded/" + uploaded_filename, 'wb') as f: f.write(content)
        mypath="Uploaded/"
        found_files=[]
        for f in listdir(mypath):
            if (f!="README.md"):
                found_files += [(f , mypath + f)]
        found_files = tuple(found_files)
        #found_files = tuple([(f , mypath + f) for f in listdir(mypath)])
        examples = ('Ice Ih', 'Examples/IceIh.pdb'),('Urea', 'Examples/urea2.pdb'), ('Ga II', 'Examples/Ga_II.vasp')
        all_files = found_files+examples
        Widget1ExampleConfiguration = interactive(chooseAndPlotConfiguration, Configuration=
                         widgets.Dropdown(
                         options=all_files,
                         description='')
                        )
        display(Widget1ExampleConfiguration)
    elif (value=='Upload'):
        display(Widget1UploadConfiguration)
        #chooseAndPlotConfiguration(uploaded_filename)
    elif (value!='Choose' and value!='Upload'):
        print("Error: keyword " + str(value) + " not recognized!")
    else:
        print("Error")
        
Widget1ExamplesAndUploadToggle = interactive(toggleChooseAndUpload, value=widgets.ToggleButtons(options=['Choose','Upload'], description=' ', disabled=False))

#display(chooseConf)

In [4]:
fractional=False

# Structure for templates
class Environments:
    def __init__(self):
        self.myindex = -1
        self.indeces = np.array([],dtype=np.int)
        self.delta = np.array([])
        self.distance =  np.array([])

globalTemplate=np.ndarray((0,),dtype=np.object)

def calculateEnvironments(lista,listb,cutoff):
    global conf, atom_types, fractional, globalTemplate
    Template = np.ndarray((0,),dtype=np.object)
    # Find all environments
    # Loop over input particles of type atom_type_1:
    myindeces=lista 
    nl = NeighborList((cutoff/2.0)*np.ones(conf.get_number_of_atoms()),skin=0.1,bothways=True)
    nl.update(conf)
    mat = conf.get_cell()
    for index in myindeces:
        neighbors, offsets = nl.get_neighbors(index)
        # Iterate over the neighbors of the current particle:
        Environment = Environments()
        Environment.myindex = index
        Environment.delta.shape = (0,3)
        for neigh, offset in zip(neighbors, offsets):
            if (neigh in listb): # Only if neighbor is in listb
                delta = conf.positions[neigh] + np.dot(offset, mat) - conf.positions[index]
                distance=np.linalg.norm(delta)
                if (distance>1.e-10):
                    if (not fractional):
                        Environment.delta = np.append(Environment.delta,np.array([[delta[0], delta[1], delta[2]]])*0.1,axis=0)
                    else:
                        Environment.delta = np.append(Environment.delta,np.array([[delta[0]/mat[0][0], delta[1]/mat[1][1], delta[2]/mat[2][2]]]),axis=0)
                    Environment.distance = np.append(Environment.distance,distance)
                    Environment.indeces = np.append(Environment.indeces,neigh)
        Template = np.append(Template,Environment)
    globalTemplate=Template


def CalculateUniqueEnvironments(Template,tolerance):
    num_of_templates=Template.shape[0]
    flag_unique=np.ones(num_of_templates)
    # Disregard empty environments
    for i in range(num_of_templates):
        if (Template[i].indeces.shape[0]==0):
            flag_unique[i]=0
    for i in trange(num_of_templates,desc="Environment 1", leave=False):
        if ( Template[i].indeces.shape[0]>0 and flag_unique[i]==1):
            for j in trange(i+1,num_of_templates,desc="Environment 2", leave=False):
                # Not empty,  unique, and same number of neighbors
                if ( Template[j].indeces.shape[0]>0 and flag_unique[j]==1 and Template[i].indeces.shape[0]==Template[j].indeces.shape[0]):
                    # Compare against all permutations
                    #for perm in tqdm(permutations(np.arange(Template[j].indeces.shape[0])),total=np.math.factorial(Template[j].indeces.shape[0]),desc="Permutations of environment", leave=None):
                    for perm in permutations(np.arange(Template[j].indeces.shape[0])):
                        p=np.array(perm)
                        # If both have the same vectors, then the second environment is not unique
                        if (np.count_nonzero(np.isclose(Template[i].delta,Template[j].delta[np.array(p),:],atol=tolerance))==Template[i].indeces.shape[0]*3 ):
                            flag_unique[j]=0
                            break
    unique_environments = np.ndarray((0,),dtype=np.object)
    for i in range(num_of_templates):
        if (flag_unique[i]==1):
            unique_environments = np.append(unique_environments,Template[i])
    return unique_environments

def CalculateUniqueEnvironmentsFast(Template,tolerance):
    # This is a fast but sometimes wrong algorithm to compare environments
    num_of_templates=Template.shape[0]
    # Sort according to x+y+z,z,y,x
    for i in range(num_of_templates):
        sortindeces=np.argsort(Template[i].delta[:,0]+Template[i].delta[:,1]+Template[i].delta[:,2],kind='stable')
        Template[i].delta = Template[i].delta[sortindeces,:]
        sortindeces=np.argsort(Template[i].delta[:,2],kind='stable')
        Template[i].delta = Template[i].delta[sortindeces,:]
        sortindeces=np.argsort(Template[i].delta[:,1],kind='stable')
        Template[i].delta = Template[i].delta[sortindeces,:]
        sortindeces=np.argsort(Template[i].delta[:,0],kind='stable')
        Template[i].delta = Template[i].delta[sortindeces,:]
    flag_unique=np.ones(num_of_templates)
    # Disregard empty environments
    for i in range(num_of_templates):
        if (Template[i].indeces.shape[0]==0):
            flag_unique[i]=0
    for i in range(num_of_templates):
        # Disregard this environment if it is empty or already not unique
        if ( Template[i].indeces.shape[0]>0 and flag_unique[i]==1):
            for j in range(i+1,num_of_templates):
                # Not empty,  unique, and same number of neighbors
                if ( Template[j].indeces.shape[0]>0 and flag_unique[j]==1 and Template[i].indeces.shape[0]==Template[j].indeces.shape[0]):
                    # If both have the same vectors, then the second environment is not unique
                    if (np.count_nonzero(np.isclose(Template[i].delta,Template[j].delta,atol=tolerance))==Template[i].indeces.shape[0]*3 ):
                        flag_unique[j]=0
    unique_environments = np.ndarray((0,),dtype=np.object)
    for i in range(num_of_templates):
        if (flag_unique[i]==1):
            unique_environments = np.append(unique_environments,Template[i])
    return unique_environments

def printEnvironmentSummaryInfo(Template):
    num_of_templates=Template.shape[0]
    print("Found " + str(num_of_templates) + " environments")

def printEnvironments(Template):
    # Print C++ syntax or plumed input syntax
    num_of_templates=Template.shape[0]
    for i in range(num_of_templates):
        env_atom_types = np.asarray(conf.get_chemical_symbols())[Template[i].indeces.astype(int)]
        env_positions = Template[i].delta*10 
        env = ase.Atoms(env_atom_types,env_positions)
        write('-', env, format='proteindatabank')
        #    for j in range(Template[i].indeces.shape[0]):
        #        print("VECTOR" + str(i+1) + "_" + str(j+1) + "=" + str(Template[i].delta[j,0]) + ","  + str(Template[i].delta[j,1]) + "," + str(Template[i].delta[j,2]))
        #        #print("DISTANCE" + str(counter+1) + "_" + str(j+1) + "=" + str(Template[i].distance[j]))

                
def plotEnv(myEnvironment): #,anglex,angley,anglez):
    #fig, ax = plt.subplots()
    #anglestr = str(anglex) + "x," + str(angley) + "y," + str(anglez) + "z"
    #print( np.append(Template[env_number].indeces.astype(int),Template[env_number].myindex) )
    #print(Template[env_number].indeces.astype(int))
    env_atom_types = np.asarray(conf.get_chemical_symbols())[myEnvironment.indeces.astype(int)] #,np.array('C')] #Template[env_number].myindex)]
    env_atom_types = np.append(env_atom_types,np.array('Si'))
    #print(env_atom_types)
    env_positions = np.vstack((myEnvironment.delta*10,np.array([0,0,0]) ) ) 
    #print(env_positions)
    env = ase.Atoms(env_atom_types,env_positions)
    #plot_atoms(env, ax, radii=0.3, rotation=(anglestr))
    #plt.show()
    v = nv.show_ase(env)
    boxy=widgets.Box([v],layout=box_layout)
    display(boxy)

def chooseEnvPlotAll(number):
    #,anglex,angley,anglez):
    global globalTemplate
    if (globalTemplate.shape[0]>0):
        plotEnv(globalTemplate[number-1]) #,anglex,angley,anglez)
    
def chooseEnvPlotUnique(number):
    #,anglex,angley,anglez):
    global uniqueEnvs
    if (uniqueEnvs.shape[0]>0):
        plotEnv(uniqueEnvs[number-1]) #,anglex,angley,anglez)
        
#def chooseEnvPlotEmpty(number,anglex,angley,anglez):
#    Environment = Environments()
#    Environment.delta.shape = (0,3)
#    plotEnv(Environment,anglex,angley,anglez)
    
def chooseEnvPlot(number): #,anglex,angley,anglez):
    global uniqueEnvs, globalTemplate, uniqueFlag
    if (uniqueFlag and uniqueEnvs.shape[0]>0):
        chooseEnvPlotUnique(number) #,anglex,angley,anglez)
    elif (not(uniqueFlag) and globalTemplate.shape[0]>0):
        chooseEnvPlotAll(number) #,anglex,angley,anglez)
    else:
        print("Error")

def envsSize():
    global uniqueEnvs, globalTemplate, uniqueFlag
    if (uniqueFlag and uniqueEnvs.shape[0]>0):
        return uniqueEnvs.shape[0]
    elif (not(uniqueFlag) and globalTemplate.shape[0]>0):
        return globalTemplate.shape[0]
    else:
        return 1

In [5]:
uniqueFlag=True
uniqueEnvs = np.ndarray((0,),dtype=np.object)
env=Environments()
env.delta.shape = (0,3)
uniqueEnvs = np.append(uniqueEnvs,env)
fastFlag=True
 
plotMyEnv = interactive(chooseEnvPlot, number=widgets.IntSlider(description='Environment:',min=1,max=envsSize(),step=1,value=0), anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))

def toggleAllVsUnique(value):
    if (value=='Unique' and uniqueEnvs.shape[0]>0  and uniqueFlag):
        plotMyEnv = interactive(chooseEnvPlotUnique, number=widgets.IntSlider(description='Environment:',min=1,max=uniqueEnvs.shape[0],step=1,value=0)) #, anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
        display(plotMyEnv)
    elif (value=='All' and globalTemplate.shape[0]>0):
        plotMyEnv = interactive(chooseEnvPlotAll, number=widgets.IntSlider(description='Environment:',min=1,max=globalTemplate.shape[0],step=1,value=0)) #, anglex=widgets.IntSlider(description='Angle x:',min=-90,max=90,step=5,value=0), angley=widgets.IntSlider(description='Angle y:',min=-90,max=90,step=5,value=0), anglez=widgets.IntSlider(description='Angle z:',min=-90,max=90,step=5,value=0))
        display(plotMyEnv)
    elif (value!='All' and value!='Unique'):
        print("Error: keyword " + str(value) + " not recognized!")
    elif (not(uniqueFlag)):
        print("Error: unique environments not requested!")
    elif (globalTemplate.shape[0]==0 or uniqueEnvs.shape[0]==0):
        print("Error: empty environments!")
    else:
        print("Error")
        
Widget3AnalyzeEnvironmentsToggle = interactive(toggleAllVsUnique, value=widgets.ToggleButtons(options=['Unique','All'], description='Choose:', disabled=False))

ToggleToRefreshMessage = widgets.Label(value='Toggle to refresh')

Widget3AnalyzeEnvironments = widgets.VBox([ToggleToRefreshMessage,Widget3AnalyzeEnvironmentsToggle])

In [6]:
def calculateEnvironmentsType(atom_type_1,atom_type_2,cutoff,tolerance):
    global conf, atom_types, globalTemplate, uniqueFlag, uniqueEnvs, fastFlag
    chemical_symbols = np.asarray(conf.get_chemical_symbols())
    lista=np.argwhere(chemical_symbols == atom_type_1).flatten()
    listb=np.argwhere(chemical_symbols == atom_type_2).flatten()
    calculateEnvironments(lista,listb,cutoff)
    if (uniqueFlag and not(fastFlag)):
        uniqueEnvs=CalculateUniqueEnvironments(globalTemplate,tolerance)
        printEnvironmentSummaryInfo(uniqueEnvs)
    elif (uniqueFlag and fastFlag):
        uniqueEnvs=CalculateUniqueEnvironmentsFast(globalTemplate,tolerance)
        printEnvironmentSummaryInfo(uniqueEnvs)
    else:
        printEnvironmentSummaryInfo(globalTemplate)

def calculateEnvironmentsString(listastring,listbstring,cutoff,tolerance):
    global conf, atom_types, globalTemplate, uniqueFlag, fastFlag
    lista=np.fromstring(listastring, dtype=int, sep=',')-1
    listb=np.fromstring(listbstring, dtype=int, sep=',')-1
    calculateEnvironments(lista,listb,cutoff)
    if (uniqueFlag and not(fastFlag)):
        uniqueEnvs=CalculateUniqueEnvironments(globalTemplate,tolerance)
        printEnvironmentSummaryInfo(uniqueEnvs)
    elif (uniqueFlag and fastFlag):
        uniqueEnvs=CalculateUniqueEnvironmentsFast(globalTemplate,tolerance)
        printEnvironmentSummaryInfo(uniqueEnvs)
    else:
        printEnvironmentSummaryInfo(globalTemplate)
    
def calculateEnvironmentsMinMaxStride(mina,maxa,stridea,minb,maxb,strideb,cutoff,tolerance):
    global conf, atom_types, globalTemplate, uniqueFlag, fastFlag
    lista=np.arange(int(mina),int(maxa)+1,int(stridea))-1
    listb=np.arange(int(minb),int(maxb)+1,int(strideb))-1
    #print(lista,listb)
    calculateEnvironments(lista,listb,cutoff)
    if (uniqueFlag and not(fastFlag)):
        uniqueEnvs=CalculateUniqueEnvironments(globalTemplate,tolerance)
        printEnvironmentSummaryInfo(uniqueEnvs)
    elif (uniqueFlag and fastFlag):
        uniqueEnvs=CalculateUniqueEnvironmentsFast(globalTemplate,tolerance)
        printEnvironmentSummaryInfo(uniqueEnvs)
    else:
        printEnvironmentSummaryInfo(globalTemplate)
    
#wvec

def toggleTypeAndIndex(value, unique, fast):
    global conf, atom_types, uniqueFlag, fastFlag
    uniqueFlag=unique
    fastFlag=fast
    if (value=='Type'):
        # Call function with widgets
        wvec = interactive(calculateEnvironmentsType, 
            atom_type_1 = widgets.Dropdown(options=atom_types, value=atom_types[0], description='Atom type 1:'),
            atom_type_2 = widgets.Dropdown(options=atom_types, value=atom_types[0], description='Atom type 2:'),
            cutoff =  widgets.FloatText(value=1,description='Cutoff:',disabled=False),
            tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    elif (value=='String'):
        # Call function with widgets
        wvec = interactive(calculateEnvironmentsString, 
                   listastring = widgets.Text(value="1,2,3", description='List A:',placeholder='Type something',disabled=False),
                   listbstring = widgets.Text(value="1,2", description='List B:',placeholder='Type something',disabled=False),
                   cutoff =  widgets.FloatText(value=2,description='Cutoff:',disabled=False),
                   tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    elif (value=='Step'):
        # Call function with widgets
        wvec = interactive(calculateEnvironmentsMinMaxStride,
                   mina = widgets.Text(value="1", description='Min A:',placeholder='Type something',disabled=False),
                   maxa = widgets.Text(value="2", description='Max A:',placeholder='Type something',disabled=False),
                   stridea = widgets.Text(value="1", description='Stride A:',placeholder='Type something',disabled=False),
                   minb = widgets.Text(value="1", description='Min B:',placeholder='Type something',disabled=False),
                   maxb = widgets.Text(value="2", description='Max B:',placeholder='Type something',disabled=False),
                   strideb = widgets.Text(value="1", description='Stride B:',placeholder='Type something',disabled=False),
                   cutoff =  widgets.FloatText(value=2,description='Cutoff:',disabled=False),
                   tolerance = widgets.FloatText(value=0.02,description='Tolerance:',disabled=False) 
        )
        display(wvec)
    else:
        print("Error")
         
Widget2DefineEnvironment = interactive(toggleTypeAndIndex, value=widgets.ToggleButtons(options=['Type','String','Step'], description='Choose:', disabled=False), 
                                       unique=widgets.Checkbox(value=True, description="Find unique environments?", disabled=False),
                                       fast=widgets.Checkbox(value=True, description="Fast algorithm", disabled=False) 
                                      )




In [7]:
def toggleAllVsUniqueForOutput(value):
    if (value=='Unique' and uniqueEnvs.shape[0]>0 and uniqueFlag):
        printEnvironments(uniqueEnvs)
    elif (value=='All' and globalTemplate.shape[0]>0):
        printEnvironments(globalTemplate)
    elif (value!='All' and value!='Unique'):
        print("Error: keyword " + str(value) + " not recognized!")
    elif (not(uniqueFlag)):
        print("Error: unique environments not requested!")
    elif (globalTemplate.shape[0]==0 or uniqueEnvs.shape[0]==0):
        print("Error: empty environments!")
    else:
        print("Error")
        
Widget4OutputEnvironmentsToggle = interactive(toggleAllVsUniqueForOutput, value=widgets.ToggleButtons(options=['Unique','All'], description='Choose:', disabled=False))

ToggleToRefreshMessage = widgets.Label(value='Toggle to refresh')

Widget4OutputEnvironments = widgets.VBox([ToggleToRefreshMessage,Widget4OutputEnvironmentsToggle])

In [8]:
#items = [widgets.Label(str(i)) for i in range(8)]
#items = [chooseConfWithTitle,wtoggle,w]
#firstColumn=widgets.VBox([chooseConfWithTitle,w])
#widgets.HBox([firstColumn,wtoggle]) #, layout=widgets.Layout(grid_template_columns="repeat(2, 600px)"))

In [9]:
children = [Widget1ExamplesAndUploadToggle,Widget2DefineEnvironment,Widget3AnalyzeEnvironments,Widget4OutputEnvironments]
tab = widgets.Tab() #layout=widgets.Layout(width='800px', height='800px'))
tab.children = children
tab.set_title(0, 'Choose configuration')
tab.set_title(1, 'Define environments')
tab.set_title(2, 'Analyze environments')
tab.set_title(3, 'Output environments')
tab



## Acknowledgments

* The app uses several python libraries, for instance [ASE](https://wiki.fysik.dtu.dk/ase/) and [NGLVIEW](https://github.com/arose/nglview).
* I am grateful to Giovanni Pizzi and Dou Du for suggesting to deploy the tool using [Binder](https://mybinder.org/)+[appmode](https://github.com/oschuett/appmode).
* This tool was developed with support of the Swiss National Science Foundation (SNSF) through an Early Postdoc.Mobility fellowship.
* I also acknowledge funding from the NCCR MARVEL funded by the SNSF and from the CSI Computational Science Center funded by the Department of Energy of the USA.

## How to cite

If you are using this tool to find environments for enhanced sampling simulations please read and cite:
* [Pablo Piaggi and Michele Parrinello, *Calculation of phase diagrams in the multithermal-multibaric ensemble*, J. Chem. Phys. 150, 244119 (2019)](https://aip.scitation.org/doi/full/10.1063/1.5102104)
